In [32]:
import pandas as pd
import numpy as np
import json
import glob
import os
import matplotlib.pyplot as plt
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
from IPython.display import Markdown, display, HTML


class StopExecution(Exception):
    def _render_traceback_(self):
        print("****   Stopping Further Execution  ****")
        pass
    
DEBUG = False

In [33]:
def createDFfrmJson(jsnfile):
    #jsnfile = 'C:/Users/mishre3/DataModelCreation/json_data/AandP_1886_AHCL.json'
    print("Working on Json file: ", jsnfile)
    with open(jsnfile, 'r') as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    df = df.dropna(axis='columns', how ='all') #drop columns if all values are NULL
    #Clean column names if they have brackets or white space (crude way, there are better ways available) 
    df.columns=df.columns.str.replace('(','')
    df.columns=df.columns.str.replace(')','')
    df.columns=df.columns.str.replace('/','')
    df.columns=df.columns.str.replace(' ','')
    print("Shape of ", jsnfile, "  ", df.shape)
    return df

def filterOnColVals(df, filterOnCol, filterOnVals):
    print("Will be filtering on column: ", filterOnCol, " for values: ", filterOnVals)
    
    if not filterOnVals: #if list is empty, No filter applied
        print("--- filterOnVals is empty, no filter applied ---")
        return df
    
    if filterOnCol in df.columns:
        if DEBUG:
            print("All non-null columns: ", sorted(df.columns))
    else:
        print(filterOnCol, "  is NOT in columns: ", sorted(df.columns) )
        raise StopExecution    

    df[filterOnCol] = df[filterOnCol].astype(str)
    allpossiblevals = sorted( df[filterOnCol].unique())
    if DEBUG:
        print("Possible Values of filterOnCol ", filterOnCol, "  are: " , allpossiblevals )
        print("Total number of rows after filter on ", filterOnVals,"   ",sum( df[filterOnCol].isin( filterOnVals ) ) )

    if sum( df[filterOnCol].isin( filterOnVals ) ):
        if DEBUG:
            print("Atleast one of: ", filterOnVals, "  is in all possible values of ", filterOnCol ,"  ", allpossiblevals )
    else:
        print("None of: ", filterOnVals, "  is in all possible values of ", filterOnCol ,"  ", allpossiblevals )
        raise StopExecution
    
    for filterval in filterOnVals:
        if filterval in allpossiblevals:
            print(filterval, " is present in filterOnVals ", filterOnVals)

    df = df[df[filterOnCol].isin( filterOnVals )]
    df = df.dropna(axis='columns', how ='all') #drop columns if all values are NULL
    if DEBUG:
        print("All non-null columns after filter: ", sorted( df.columns) )    
    print("Shape after filter on ", filterOnCol, "  for values: " ,  filterOnVals, "  is  ", df.shape)    
    return df


def keepColsOfDF(df, keepCols):
    df.drop(df.columns.difference(keepCols), 1, inplace=True)
    df = df[keepCols]
    print("Shape after filter and keepCols: ", df.shape)
    return df

def createDateCol(df, frmTimeStampCol='timestamp'):
    newtimestamp = frmTimeStampCol + 'D'
    df[newtimestamp] = pd.to_datetime(df[frmTimeStampCol]) 
    df['Date'] = pd.to_datetime( df[newtimestamp].dt.date )
    return df

def createTimeCol(df, frmTimeStampCol='timestamp'):
    newtimestamp = frmTimeStampCol + 'T'
    df[newtimestamp] = pd.to_datetime(df[frmTimeStampCol]) 
    df['Time'] = df[newtimestamp].dt.time
    df['Time'] = df['Time'].astype(str)
    return df
    
def getLables(bins):
    bins[0] = bins[0]-1 
    labels = [ str( str(bins[i]+1) + "-" + str(bins[i+1]) ) for i in range( 0, len(bins)-1 ) ]
    return labels


def createGetDate(df):
    df = createDateCol(df)
    alldates = pd.to_datetime ( sorted(df['Date'].unique()) )
    alldates = [str(i.date()) for i in alldates]
    #print("All Dates: \n",  alldates)
    return alldates


def filterOnTimeRange(df, startTime='06:00:00', endTime='11:00:00'):
    df = df[ df['Time'] >= startTime ]
    df = df[ df['Time'] < endTime ]
    return df

In [42]:
#jsnfile = 'C:/Users/mishre3/DataModelCreation/json_data/AandP_1886_AHCL.json'
jsnfile = 'C:/Users/mishre3/DataModelCreation/json_data/670_Jan1to15.json'

df = createDFfrmJson(jsnfile)
filterOnCol = "className"
#filterOnVals = ["GlucoseSensorData"]
filterOnVals = ["GlucoseSensorData","GlucoseSensorDataHigh","GlucoseSensorDataLow"]
df = filterOnColVals(df, filterOnCol, filterOnVals)

df = createTimeCol(df)
df = createDateCol(df)
keepCols = ['timestamp', 'Date', 'Time', 'amount']
df = keepColsOfDF(df, keepCols)

print(df.head())
print(df.dtypes)

Working on Json file:  C:/Users/mishre3/DataModelCreation/json_data/670_Jan1to15.json
Shape of  C:/Users/mishre3/DataModelCreation/json_data/670_Jan1to15.json    (40411, 227)
Will be filtering on column:  className  for values:  ['GlucoseSensorData', 'GlucoseSensorDataHigh', 'GlucoseSensorDataLow']
GlucoseSensorData  is present in filterOnVals  ['GlucoseSensorData', 'GlucoseSensorDataHigh', 'GlucoseSensorDataLow']
Shape after filter on  className   for values:  ['GlucoseSensorData', 'GlucoseSensorDataHigh', 'GlucoseSensorDataLow']   is   (3366, 20)
Shape after filter and keepCols:  (3366, 4)
                         timestamp       Date      Time  amount
36757  2021-01-02T21:24:42.00+0000 2021-01-02  21:24:42   155.0
36758  2021-01-02T21:29:42.00+0000 2021-01-02  21:29:42   159.0
36759  2021-01-02T21:34:42.00+0000 2021-01-02  21:34:42   157.0
36760  2021-01-02T21:39:42.00+0000 2021-01-02  21:39:42   154.0
36761  2021-01-02T21:44:42.00+0000 2021-01-02  21:44:42   151.0
timestamp        

In [43]:
fileNameExc = "C:\\Users\\mishre3\\DataModelCreation\\PSA\\670_Jan1to15_Data.xlsx"
#df.to_excel (fileNameExc, index = False, header=True)
segment = ["Breakfast", "Lunch", "Dinner", "Overnight"]
startTime = ['06:00:00', '11:00:00', '16:00:00', '22:00:00', '00:00:00'] #Will be included 
endTime =   ['11:00:00', '16:00:00', '22:00:00', '23:59:59', '06:00:00'] #Will not be included
    
for myseg in range(0, len(segment)):
    fileNameExc = "C:\\Users\\mishre3\\DataModelCreation\\PSA\\" + segment[myseg] + "Data.xlsx"
    fileNameCsv = "C:\\Users\\mishre3\\DataModelCreation\\PSA\\" + segment[myseg] + "Data.csv"
    if myseg==3:
        dfx1 = filterOnTimeRange(df, startTime[myseg], endTime[myseg])
        dfx2 = filterOnTimeRange(df, startTime[myseg+1], endTime[myseg+1])
        dfx = pd.concat([dfx1, dfx2])
    else:
        dfx = filterOnTimeRange(df, startTime[myseg], endTime[myseg])
    
    print("TimeStamp Max/Min  \n"  , dfx['timestamp'].min(),  dfx['timestamp'].max() )
    print("Time Max/Min  \n"  , dfx['Time'].min(),  dfx['Time'].max() )
    #dfx.to_excel (fileNameExc, index = False, header=True)
    dfx.to_csv(fileNameCsv, index=False, header=True)
    
data = df

print(df.head())
print(df.dtypes)

TimeStamp Max/Min  
 2021-01-03T06:04:43.00+0000 2021-01-15T10:56:10.00+0000
Time Max/Min  
 06:01:04 10:59:55
TimeStamp Max/Min  
 2021-01-03T11:04:43.00+0000 2021-01-15T15:56:12.00+0000
Time Max/Min  
 11:00:58 15:59:57
TimeStamp Max/Min  
 2021-01-02T21:24:42.00+0000 2021-01-15T16:31:12.00+0000
Time Max/Min  
 16:00:54 21:59:53
TimeStamp Max/Min  
 2021-01-02T22:04:42.00+0000 2021-01-15T05:56:08.00+0000
Time Max/Min  
 00:00:56 23:59:53
                         timestamp       Date      Time  amount
36757  2021-01-02T21:24:42.00+0000 2021-01-02  21:24:42   155.0
36758  2021-01-02T21:29:42.00+0000 2021-01-02  21:29:42   159.0
36759  2021-01-02T21:34:42.00+0000 2021-01-02  21:34:42   157.0
36760  2021-01-02T21:39:42.00+0000 2021-01-02  21:39:42   154.0
36761  2021-01-02T21:44:42.00+0000 2021-01-02  21:44:42   151.0
timestamp            object
Date         datetime64[ns]
Time                 object
amount              float64
dtype: object


In [58]:
bins = [40, 54, 69, 180, 248, 400] 
lables = getLables(bins)
lastlabel = lables[-1]

def getPercentageDF(df, OverAll=""):
    if len(OverAll):
        df['Date'] = min ( df['Date'] )    
        
    df = df.set_index('Date')
    df.index = pd.to_datetime(df.index)
    df = df.pivot_table(values='amount', index=df.index, columns='TIF', aggfunc='size')
    df = df.sort_values(by=['Date'], ascending=False)
    
    df.columns = df.columns.astype(str)
    rCols =  list(df.columns.values) 
    df = df.div(df.sum(1), 0).mul(100).round(0).assign(Sum=lambda df: df.sum(axis=1))
    df = df.reset_index()

    df[lastlabel] = df.apply(lambda x: x[lastlabel]-1 if x['Sum']==101 else x[lastlabel], axis=1)
    df[lastlabel] = df.apply(lambda x: x[lastlabel]+1 if x['Sum']==99 else x[lastlabel], axis=1)
    
    dropCols = ['Sum']
    df.drop(dropCols, axis=1, inplace=True)
    df = df.fillna(0)
    df['Date'] = pd.to_datetime( df['Date'] ).dt.date
    if len(OverAll):
        df['Date'] = OverAll        
    return df


for myseg in range(0, len(segment)):
    fileNameCsv = "C:\\Users\\mishre3\\DataModelCreation\\PSA\\TIF_" + segment[myseg] + "Data.csv"
    if myseg==3:
        dfx1 = filterOnTimeRange(df, startTime[myseg], endTime[myseg])
        dfx2 = filterOnTimeRange(df, startTime[myseg+1], endTime[myseg+1])
        dfx = pd.concat([dfx1, dfx2])
    else:
        dfx = filterOnTimeRange(df, startTime[myseg], endTime[myseg])

    keepCols = ['Date', 'amount']
    dfx = keepColsOfDF(dfx, keepCols)
    dfx['TIF'] = pd.cut(dfx['amount'], bins, labels=lables)
    dfx = getPercentageDF(dfx, segment[myseg]).reset_index()
    dfx.drop('index', axis=1, inplace=True)
    dfx.rename(columns = {'Date':'Segment'}, inplace = True)
    dfx.name = None
    #print(dfx.columns)
    #print(segment[myseg], "\n",  dfx)
    dfx.to_csv(fileNameCsv, index=False, header=True)
    
data = df

Shape after filter and keepCols:  (686, 2)
Shape after filter and keepCols:  (779, 2)
Shape after filter and keepCols:  (841, 2)
Shape after filter and keepCols:  (1060, 2)


In [37]:
df = data
df['LT54'] =       df['amount'].apply(lambda x: True if x < 54 else False)
df['LT70UPTO54'] = df['amount'].apply(lambda x: True if (x < 70) and (x >= 54) else False)

keepCols = ['Time', 'amount', 'LT54', 'LT70UPTO54']
df = keepColsOfDF(df, keepCols)
df["Time"] = pd.to_datetime(df["Time"])
df.set_index("Time", inplace=True)

aggdict = {"amount":"count", "LT54":"sum", "LT70UPTO54":"sum"}
dfg = df.groupby(pd.Grouper(freq='5Min',closed='right',label='right')).agg(aggdict).reset_index()

dfg["LT54_percent"] = round( ( dfg["LT54"] * 100 / dfg['amount'] ), 2 )
dfg["LT70UPTO54_percent"] = round( ( dfg["LT70UPTO54"] * 100 / dfg['amount'] ), 2 )

dfg["Time"] = pd.to_datetime(dfg["Time"])
dfg = createTimeCol(dfg, "Time")
keepCols = ['Time','LT54_percent','LT70UPTO54_percent']
dfg = keepColsOfDF(dfg, keepCols)
dfg

Shape after filter and keepCols:  (3366, 4)
Shape after filter and keepCols:  (288, 3)


,Time,LT54_percent,LT70UPTO54_percent
0,00:05:00,0.0,7.69
1,00:10:00,0.0,7.69
2,00:15:00,0.0,16.67
3,00:20:00,0.0,16.67
4,00:25:00,0.0,16.67
...,...,...,...
283,23:40:00,0.0,7.69
284,23:45:00,0.0,7.69
285,23:50:00,0.0,7.69
286,23:55:00,0.0,7.69


In [38]:
df = data
df['GT250'] =      df['amount'].apply(lambda x: True if x > 250 else False)
df['GT180LT250'] = df['amount'].apply(lambda x: True if (x > 180) and (x <= 250) else False)

keepCols = ['Time', 'amount', 'GT250', 'GT180LT250']
df = keepColsOfDF(df, keepCols)
df["Time"] = pd.to_datetime(df["Time"])
df.set_index("Time", inplace=True)

aggdict = {"amount":"count", "GT250":"sum", "GT180LT250":"sum"}
dfg = df.groupby(pd.Grouper(freq='5Min',closed='right',label='right')).agg(aggdict).reset_index()

dfg["GT250_percent"] = round( ( dfg["GT250"] * 100 / dfg['amount'] ), 2 )
dfg["GT180LT250_percent"] = round( ( dfg["GT180LT250"] * 100 / dfg['amount'] ), 2 )

dfg["Time"] = pd.to_datetime(dfg["Time"])
dfg = createTimeCol(dfg, "Time")
keepCols = ['Time','GT250_percent','GT180LT250_percent']
dfg = keepColsOfDF(dfg, keepCols)
dfg

Shape after filter and keepCols:  (3366, 4)
Shape after filter and keepCols:  (288, 3)


,Time,GT250_percent,GT180LT250_percent
0,00:05:00,0.0,15.38
1,00:10:00,0.0,7.69
2,00:15:00,0.0,0.00
3,00:20:00,0.0,0.00
4,00:25:00,0.0,8.33
...,...,...,...
283,23:40:00,0.0,7.69
284,23:45:00,0.0,7.69
285,23:50:00,0.0,7.69
286,23:55:00,0.0,7.69


In [39]:
df = data

keepCols = ['Time', 'amount']
df = keepColsOfDF(df, keepCols)
df["Time"] = pd.to_datetime(df["Time"])
df.set_index("Time", inplace=True)

aggdict = {"amount":"mean"}
dfg = df.groupby(pd.Grouper(freq='5Min',closed='right',label='right')).agg(aggdict).reset_index()
dfg.rename(columns = {'amount':'mean'}, inplace = True)

dfg["10_percent"] = round( dfg["mean"] * 0.10 , 2 )
dfg["25_percent"] = round( dfg["mean"] * 0.25 , 2 )
dfg["75_percent"] = round( dfg["mean"] * 0.75 , 2 )
dfg["90_percent"] = round( dfg["mean"] * 0.90 , 2 )

dfg["Time"] = pd.to_datetime(dfg["Time"])
dfg = createTimeCol(dfg, "Time")
keepCols = ['Time','mean','10_percent','25_percent','75_percent','90_percent']
dfg = keepColsOfDF(dfg, keepCols)
dfg

Shape after filter and keepCols:  (3366, 2)
Shape after filter and keepCols:  (288, 6)


,Time,mean,10_percent,25_percent,75_percent,90_percent
0,00:05:00,131.846154,13.18,32.96,98.88,118.66
1,00:10:00,131.692308,13.17,32.92,98.77,118.52
2,00:15:00,127.416667,12.74,31.85,95.56,114.68
3,00:20:00,128.250000,12.82,32.06,96.19,115.42
4,00:25:00,128.833333,12.88,32.21,96.62,115.95
...,...,...,...,...,...,...
283,23:40:00,132.538462,13.25,33.13,99.40,119.28
284,23:45:00,132.000000,13.20,33.00,99.00,118.80
285,23:50:00,132.384615,13.24,33.10,99.29,119.15
286,23:55:00,131.692308,13.17,32.92,98.77,118.52
